# The Alexander-Whitney, Eilenberg-Zilber, and Shih maps

## Abstract

We describe functions modeling the Alexander-Whitney map, the Eilenberg-Zilber map, and a chain homotopy from their non-trivial composition to the identity. These are part of the `simplicial_operators` package.

In [ ]:
pip install simplicial_operators

In [ ]:
from simplicial_operators import Operator

In [ ]:
# to be absorved

from simplicial_operators import Operator

# Absorb display into the package
def display(multiop):
    '''tool to aid visualization of linear multioperators'''
    if isinstance(multiop, Operator):
        return str(multiop)
    if isinstance(multiop, tuple):
        return tuple(display(op) for op in multiop)
    if isinstance(multiop, set):
        return set(display(mop) for mop in multiop)
    else:
        raise TypeError('expected either: Operator, tuple of Operators, or set of these tuples')
        
# Absorb into the package
def is_degenerate(multiop):
    '''determines if a multioperator is degenerate'''
    if isinstance(multiop, Operator):
        return multiop.is_degeenrate
    if isinstance(multiop, tuple):
        deg = set(multiop[0].deg_maps)
        for op in multiop:
            deg = deg.intersection(set(op.deg_maps))
        return bool(deg)
    else:
        raise TypeError('expected either Operator or tuple of Operators')
        
def is_nondegenerate(multiop):
    return not is_degenerate(multiop)

## Preliminaries

Throughout this notebook all algebraic constructions are taken over the field $\mathbb F_2$ with two elements. 

We assume familiarity with the following notions discussed in this notebook LINK:

1. Simplicial set
2. Simplicial operator
3. Chain complex 
4. Tensor product
5. Normalized chains $N_*$
6. Linear simplicial multioperator



## Products and AW and EZ maps

The product of two simplicial sets $X$ and $Y$ is defined by
\begin{equation}
(X \times Y)_n = X_n \times Y_n,
\end{equation}

\begin{equation}
d_i(x \times y) = d_i x \times d_i y,
\end{equation}

\begin{equation}
s_i(x \times y) = s_i x \times s_i y.
\end{equation}

In general
$$
N_*(X \times Y) \neq N_*(X) \otimes N_*(Y)
$$

but the Alexander-Whitney and Eilenberg-Zilber maps form a chain homotopy equivalence $N_*(X \times Y) \sim N_*(X) \otimes N_*(Y)$. I.e., there exist chain homotopies between $AWEZ$, $EZAW$ and the respective identities. It turns out that $AWEZ$ is equal to the identity and that a canonical chain homotopy, known as the Shih map, can be given for the other composition. 

Below we describe these three maps and their models in the `simplicial_operators` package.

### The **Alexander-Whitney map**
\begin{equation*}
    AW: N_*(X \times Y) \to N_*(X) \otimes N_*(Y)
\end{equation*}
is defined for $x \times y \in N_*(X \times Y)_n$ by
\begin{equation*}
    AW(x \times y) = \sum_{i=0}^n d_{i+1} \cdots d_n\, x \otimes d_0 \cdots d_{i-1}\, y.
\end{equation*}




In [ ]:
from simplicial_operators import alexander_whitney as aw

#### Example
For the map
\begin{equation*}
AW : N_*(\Delta^1 \times \Delta^2) \to N_*(\Delta^1) \otimes N_*(\Delta^2)
\end{equation*}
we verify that
\begin{equation*}
AW([0,0,0,1] \times [0,1,2,2]) = 
[0] \otimes [0,1,2,2] + [0,0] \otimes [1,2,2] + 
[0,0,0] \otimes [2,2] +[0,0,0,1] \otimes [2].
\end{equation*}

In [ ]:
{(biop[0]([0,0,0,1]), biop[1]([0,1,2,2])) for biop in aw(3).values()}

### The **Eilenberg-Zilber map**

\begin{equation*}
    EZ: N_*(X) \otimes N_*(Y) \to N_*(X \times Y)
\end{equation*}
is defined for $x \otimes y \in N_p(X) \otimes N_q(Y)$ by
\begin{equation*}
    EZ(x \otimes y) = 
    \sum s_{v_p} \cdots s_{v_1}\, x \times s_{w_q} \cdots s_{w_1}\, y
\end{equation*}
where the sum is over all ordered partitions 
\begin{equation*}
\big( \{v_1 < \cdots < v_p\},\ \{w_1 < \cdots < w_q\} \big)
\end{equation*}	
of $\{0,\dots,p+q-1\}$. An **ordered partition** of a set $S$ is a tuple of disjoint subsets of $S$ whose union is $S$.

In [ ]:
from simplicial_operators import eilenberg_zilber as ez

#### Example
For the map
\begin{equation*}
EZ : N_*(\Delta^1) \otimes N_*(\Delta^2) \to N_*(\Delta^1 \times \Delta^2)
\end{equation*}

we verify that

\begin{equation*}
EZ([0,1] \otimes [0,1,2]) = [0,0,0,1] \times [0,1,2,2] + [0,0,1,1] \times [0,1,1,2] + [0,1,1,1] \times [0,0,1,2].
\end{equation*}

In [ ]:
{(biop[0]([0,1]), biop[1]([0,1,2])) for biop in ez(1,2)}

#### Example

The map
\begin{equation*}
EZAW : C_\bullet(\Delta^2 \times \Delta^2) \to C_\bullet(\Delta^2 \times \Delta^2)
\end{equation*}

is not equal to the identity. We verify that

\begin{equation*}
EZAW([0,1,2] \otimes [0,1,2]) =  [0, 0, 0] \otimes [0, 1, 2] + [0, 0, 1] \otimes [1, 2, 2] + [0, 1, 1] \otimes [1, 1, 2] + [0, 1, 2] \otimes  [2, 2, 2].
\end{equation*}

In [ ]:
def ezaw(n):
    e_z, a_w = ez(n), aw(n)
    answer = set()
    for i in range(n+1):
        answer ^= {(biop[0].compose(a_w[(-i,-n+i)][0]),
                    biop[1].compose(a_w[(-i,-n+i)][1]))
                    for biop in e_z[(i,n-i)]}
        
    return answer

{(biop[0]([0,1,2]), biop[1]([0,1,2])) for biop in ezaw(2)}

### The Shih homotopy $SHI: EZAW \Rightarrow id$

The composition $EZAW$ is chain homotopic to the identity via the map

\begin{equation*}
SHI_n : N_n(X \times Y) \to N_{n+1}(X \times Y)
\end{equation*}
recursively defined by 
\begin{equation*}
SHI_n = 
\begin{cases} 
0 & n = 0 \\
SHI'_{n-1} + EZAW ' (s_0 \otimes s_0) & n > 0
\end{cases}
\end{equation*}
where 
\begin{equation*}
\big( s_{v_{l}} \cdots s_{v_1} d_{i_1} \cdots d_{i_p} \big) ' = s_{v_{l}+1} \cdots s_{v_1+1} d_{i_1+1} \cdots d_{i_p+1}.
\end{equation*}

In [ ]:
from simplicial_operators import shih

#### Example

For the map
\begin{equation*}
SHI : C_\bullet(\Delta^2 \times \Delta^2) \to C_\bullet(\Delta^2 \times \Delta^2)
\end{equation*}

we verify that

\begin{equation*}
SHI([0,1] \times [0,1]) = [0, 0, 1] \times [0, 1, 1]
\end{equation*}
and

\begin{equation*}
SHI([0,1,2] \times [0,1,2]) = 
[0, 0, 0, 1] \times [0, 1, 2, 2] + 
[0, 0, 1, 1] \times [0, 1, 1, 2] + 
[0, 0, 1, 2] \times [0, 2, 2, 2] +
[0, 1, 1, 2] \times [0, 1, 2, 2]  
\end{equation*}

In [ ]:
answer1 = set()
for biop in shih(1):
    if is_nondegenerate(biop):
        answer1 ^= {(biop[0]([0,1]), biop[1]([0,1]))}

print(answer1,'\n\nand\n')

answer2 = set()
for biop in shih(2):
    if is_nondegenerate(biop):
        answer2 ^= {(biop[0]([0,1,2]), biop[1]([0,1,2]))}

print(answer2)

### Verifying $AWEZ = id$ and $EZAW \sim id$ 

We define functions that verify that the composition

\begin{equation}
N_*(X) \otimes N_*(Y) 
\stackrel{EZ}{\longrightarrow} N_{*}(X \times Y)
\stackrel{AW}{\longrightarrow} N_*(X) \otimes N_*(Y)
\end{equation}

is the identity and that $SHI$ is a chain homotopy between the identity and the composition

\begin{equation}
N_*(X \times Y) 
\stackrel{AW}{\longrightarrow} N_*(X) \otimes N_*(Y)
\stackrel{EZ}{\longrightarrow} N_{*}(X \times Y).
\end{equation}

I.e.
\begin{equation}
\partial \, SHI + SHI \, \partial = id + EZAW.
\end{equation}

In [ ]:
from collections import Counter
from itertools import product

def is_awez_equal_to_id(p,q):
    '''returns True if the composition AWEZ restricted to bidegree 
    (p,q) is the identity and False otherwise'''
    
    comps = {biop1.compose(biop2) for biop1, biop2 in 
                                      product(aw(p+q).values(), ez(p,q))}
    
    non_deg_comps = {biop for biop in comps if  not biop.op1.is_degenerate 
                                            and not biop.op2.is_degenerate}
    
    if [non_deg_comps].sort() is [Bioperator()].sort():
        return True
    else:
        return False

def boundary(n):
    '''returns the bilinear operators defining the boundary in N_n(X x Y)'''
    
    return {Bioperator(face_maps1 = [i], 
                       face_maps2 = [i]) for i in range(n+1)}

def is_ezaw_homotopic_to_id(n):
    '''returns True if in degree n the composition EZAW is chain homotopic to the 
    identity via SHI and False if not'''
    
    # \partial \circ SHI
    a = [biop1.compose(biop2) for biop1, biop2 in product(boundary(n+1), shih(n))]
    # SHI \circ \partial
    b = [biop1.compose(biop2) for biop1, biop2 in product(shih(n-1), boundary(n))]
    # EZAW
    c = list(ezaw(n))
    # id
    d = [Bioperator()]
    
    answer = Counter([str(biop) for biop in (a+b+c+d) if not biop.is_degenerate])
    
    # cheking all coefficients are 0 mod 2
    if {0} == {i%2 for i in answer.values()}:
        return True 
    else: 
        return False

In [ ]:
p, q, n = 3,5,7 

print(f'is AWEZ equal to the identity in bidegree {p,q}?')

if is_awez_equal_to_id(p,q):
    print('Yes!')
    
else:
    print('No!')
    
print(f'is AWEZ chain homotopic to the identity in degree {n}?')

if is_ezaw_homotopic_to_id(n):
    print('Yes!')
    
else:
    print('No!')

## TO-DO

Add the Frobenius Structure as described [here](https://ncatlab.org/nlab/show/Frobenius+monoidal+functor)

Write better verifications of $AWEZ = id$ and $EZAW \sim id$